### Prepping Data Challenge: Prep Air In-Flight Purchases (week 14)

For this week's challenge, there is a selection of different data sources that needs to be combined them to answer some questions that will help in the understanding of some purchasing patterns on the flights.

#### Requirement:
 - Input the Data
 - Assign a label for where each seat is located. 
   They are assigned as follows:
     - A & F - Window Seats
     - B & E - Middle Seats
     - C & D - Aisle Seats 

 - Combine the Seat List and Passenger List tables. 

 - Parse the Flight Details so that they are in separate fields  

 - Calculate the time of day for each flight. 
   They are assigned as follows: 
    - Morning - Before 12:00 
    - Afternoon - Between 12:00 - 18:00
    - Evening - After 18:00

 - Join the Flight Details & Plane Details to the Passenger & Seat tables. We should be able to identify what rows are Business or Economy Class for each flight. 

 - Answer the following questions: 
   1. What time of day were the most purchases made? We want to take a look at the average for the flights within each time period. 
   2. What seat position had the highest purchase amount? Is the aisle seat the highest earner because it's closest to the trolley?
   3. As Business Class purchases are free, how much is this costing us? 

### Input the data

In [1]:
import pandas as pd
import numpy as np

In [2]:
with pd.ExcelFile('WK14-Input.xlsx') as xlsx:
    Passenger = pd.read_excel(xlsx, 'Passenger List')
    Seat = pd.read_excel(xlsx, 'SeatList')
    Flight = pd.read_excel(xlsx, 'FlightDetails')
    Plane = pd.read_excel(xlsx, 'PlaneDetails')

In [ ]:
#Passenger.head()
#Seat.head()
#Flight.head()
#Plane.head()

In [3]:
Passenger.columns

Index(['first_name', 'last_name', 'passenger_number', 'flight_number',
       'purchase_amount', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')

In [4]:
Passenger = Passenger[['first_name', 'last_name', 'passenger_number', 'flight_number',
       'purchase_amount']]

### Assign a label for where each seat is located

In [5]:
Seat = pd.melt(Seat, id_vars='Row', value_vars=Seat.columns[1:], var_name = 'Seat Position', value_name = 'passenger_number')

In [6]:
label = {'A':'Window', 'F':'Window', 'B':'Middle', 'E':'Middle', 'C':'Aisle', 'D':'Aisle'}
Seat['Seat Position'] = Seat['Seat Position'].map(label)

### Combine the Seat List and Passenger List tables. 

In [7]:
passenger_seat = pd.merge(Passenger, Seat, how='left', on=['passenger_number'])

### Parse the Flight Details so that they are in separate fields 

In [8]:
flight = Flight.iloc[:, 0].str.replace('[\[\]]', '').str.split('|', expand=True)
flight.columns = Flight.columns[0].replace('[', '').replace(']', '').split('|')
flight['FlightID'] = flight['FlightID'].astype(int)

C:\ProgramData\Anaconda3\envs\Tetteyanalysis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


### Calculate the time of day for each flight. 

In [9]:
flight['Time of Day'] = np.where(flight['DepTime'] < '12:00:00', 'Morning',
                                  np.where(flight['DepTime'] <= '18:00:00', 'Afternoon', 'Evening'))

### Join the Flight Details & Plane Details to the Passenger & Seat tables. We should be able to identify what rows are Business or Economy Class for each flight.

In [10]:
flight_plane = pd.merge(flight, Plane, left_on = 'FlightID', right_on='FlightNo.', how='left')

In [11]:
Combined = pd.merge(passenger_seat, flight_plane, left_on = 'flight_number', right_on = 'FlightNo.', how = 'left')

In [12]:
Combined.head()

,first_name,last_name,passenger_number,flight_number,purchase_amount,Row,Seat Position,FlightID,DepAir,ArrAir,DepDate,DepTime,Time of Day,FlightNo.,Business Class
0,Jerrylee,Rein,1,1,48.29,1,Window,1,LHR,SEA,2020-10-08,14:53:00,Afternoon,1,1-5
1,Forester,Iashvili,2,1,0.00,1,Middle,1,LHR,SEA,2020-10-08,14:53:00,Afternoon,1,1-5
2,Shaun,Sherwill,3,1,0.00,1,Aisle,1,LHR,SEA,2020-10-08,14:53:00,Afternoon,1,1-5
3,Werner,Basile,4,1,58.21,1,Aisle,1,LHR,SEA,2020-10-08,14:53:00,Afternoon,1,1-5
4,Kerwinn,Skillen,5,1,41.96,1,Middle,1,LHR,SEA,2020-10-08,14:53:00,Afternoon,1,1-5


In [13]:
#Parse the row types
Combined[['min', 'max']] = Combined['Business Class'].str.split('-', expand=True).astype(int)
Combined.drop(columns=['Business Class'], inplace=True)

In [14]:
# identify the row type (Business Class or Economy)
Combined['Business Class'] = np.where((Combined['Row'] >= Combined['min'])
                                   & (Combined['Row'] <= Combined['max']),
                                   'Business Class', 'Economy')

In [15]:
#check for missing values
Combined.isna().sum()

first_name          0
last_name           0
passenger_number    0
flight_number       0
purchase_amount     0
Row                 0
Seat Position       0
FlightID            0
DepAir              0
ArrAir              0
DepDate             0
DepTime             0
Time of Day         0
FlightNo.           0
min                 0
max                 0
Business Class      0
dtype: int64

###  Answer the following questions: 
   1. What time of day were the most purchases made? We want to take a look at the average for the flights within each time period. 
   2. What seat position had the highest purchase amount? Is the aisle seat the highest earner because it's closest to the trolley?
   3. As Business Class purchases are free, how much is this costing us? 

In [16]:
#question 1
q1 = Combined[Combined['Business Class'] != 'Business Class']\
              .groupby(['flight_number', 'Time of Day'])['purchase_amount'].sum()\
              .groupby('Time of Day').mean().reset_index()\
              .sort_values(by='purchase_amount', ascending=False)
q1.rename(columns={'purchase_amount' : 'Avg per Flight'}, inplace=True)
q1['Avg per Flight'] = q1['Avg per Flight'].round(2)
q1['Rank'] = q1['Avg per Flight'].rank(ascending=False).astype(int)

In [17]:
#question 2
q2 = Combined[Combined['Business Class'] != 'Business Class']\
         .groupby('Seat Position')['purchase_amount'].sum().reset_index()\
         .sort_values(by='purchase_amount', ascending=False)
q2.rename(columns={'purchase_amount' : 'Purchase Amount'}, inplace=True)
q2['Rank'] = q2['Purchase Amount'].rank(ascending=False).astype(int)

In [18]:
#question 3
q3 = Combined.groupby('Business Class')['purchase_amount'].sum().reset_index()\
             .sort_values(by='purchase_amount', ascending=False)
q3.rename(columns={'purchase_amount' : 'Purchase Amount'}, inplace=True)
q3['Rank'] = q3['Purchase Amount'].rank(ascending=False).astype(int)

### Write the files to Excel

In [19]:
with pd.ExcelWriter('WK14-output.xlsx') as w:
    q1[['Rank', 'Time of Day', 'Avg per Flight']]\
        .to_excel(w, sheet_name='Time of Day', index=False)
    q2[['Rank', 'Seat Position', 'Purchase Amount']]\
        .to_excel(w, sheet_name='Seat Position', index=False)
    q3[['Rank', 'Business Class', 'Purchase Amount']]\
        .to_excel(w, sheet_name='Business or Economy', index=False)